In [17]:
import pandas as pd
import requests
from mitrecve import crawler
import random
from pathlib import Path
import threading
from tqdm import tqdm
tqdm.pandas()

In [18]:
# read original labelled file to match
data_path = Path.cwd().parent.joinpath('data', 'label_data.csv').as_posix()
labelled_df = pd.read_csv(data_path, index_col=0)

# get all malicious packages
malicious_df = labelled_df[labelled_df['Label'] == 1]
# keep necessary columns
mal_df = malicious_df[['Name', 'Version', 'Label']].copy()

In [19]:
# match mitre for CVE Descriptions
def mitre_cve_api(package, timeout=300):
    result = {}

    def target():
        try:
            cve_simple = crawler.get_main_page(package)
            result['data'] = crawler.get_cve_detail(cve_simple)
        except Exception as e:
            result['error'] = e

    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        raise TimeoutError(f"Timeout querying {package} after {timeout} seconds.")
    if 'error' in result:
        raise result['error']
    return result.get('data', {})


def extract_random_desc(package_name, timeout=300):
    try:
        result = mitre_cve_api(package_name, timeout=timeout)
        descs = [entry.get('DESC', '') for entry in result.values() if 'DESC' in entry]
        return random.choice(descs) if descs else ''
    except Exception as e:
        print(f"⚠️ Error querying {package_name}: {e}")
        return ''

In [21]:
mal_df['Desc'] = mal_df['Name'].progress_apply(lambda x: extract_random_desc(x, timeout=300))


 12%|█▏        | 279/2259 [08:34<49:45:37, 90.47s/it]

⚠️ Error querying @primeo/shell: Timeout querying @primeo/shell after 300 seconds.


 16%|█▌        | 358/2259 [12:12<26:50:29, 50.83s/it]

⚠️ Error querying @m365-admin/utilities: list index out of range


 23%|██▎       | 527/2259 [18:54<43:29:59, 90.42s/it]

...

 97%|█████████▋| 2199/2259 [2:03:35<1:30:33, 90.55s/it]

⚠️ Error querying @bitkub-moonshot/common: Timeout querying @bitkub-moonshot/common after 300 seconds.


100%|█████████▉| 2248/2259 [2:09:08<16:34, 90.42s/it]  

⚠️ Error querying @rustore-web/config: Timeout querying @rustore-web/config after 300 seconds.


100%|██████████| 2259/2259 [2:09:15<00:00,  3.43s/it]


In [22]:
# save the results
mal_df.to_csv('mal_packages_with_desc.csv', index=False)